In [5]:
!pip install transformers accelerate torch


  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [6]:
pip install huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [8]:
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load JSON files
with open("/Users/bisman/Documents/ECS 260/Project github/CodeRefineAI/llm/RQ2/Inefficient_reasoning.json", "r") as f:
    reasoning_data = json.load(f)

with open("/Users/bisman/Documents/ECS 260/Project github/CodeRefineAI/dataset/balanced_samples.json", "r") as f:
    codes_data = json.load(f)

# Convert reasoning_data into a dictionary for easy lookup
reasoning_dict = {item["question_id"]: item for item in reasoning_data}

# Load Llama-3.2-1B model and tokenizer
MODEL_NAME = "meta-llama/Llama-3.2-1B"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Function to process and generate optimized code
def get_optimized_code():
    optimized_solutions = []
    api_calls = 0  # Track API call count

    for index, code_entry in enumerate(codes_data, start=1):
        qid = code_entry["question_id"]

        # Check if question_id exists in reasoning_dict
        if qid not in reasoning_dict:
            print(f"Skipping question_id {qid} (not found in reasoning data)")
            continue

        # Ensure "reason" key exists
        if "reason" not in reasoning_dict[qid]:
            print(f"Skipping question_id {qid} (missing 'reason' key)")
            continue

        # Ensure "runtime_inefficient_codes" exists and is not empty
        if "runtime_inefficient_codes" not in code_entry or not code_entry["runtime_inefficient_codes"]:
            print(f"Skipping question_id {qid} (missing or empty 'runtime_inefficient_codes')")
            continue

        # Get reasoning and first inefficient code
        reasoning = reasoning_dict[qid]["reason"]
        inefficient_code = code_entry["runtime_inefficient_codes"][0]["code"]

        # Create prompt
        prompt = f"""
        The following Python code seems to be inefficient:
        
        ```python
        {inefficient_code}
        ```
        
        Reason for inefficiency:
        {reasoning}
        
        Please rewrite the code based on the provided reason, and only return the corrected code, no explanation. 
        Use Python 3.8 syntax, and ensure the entire solution is enclosed in a `class Solution`, 
        maintaining the same function name as in the input code.
        """

        try:
            # Tokenize and generate response
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            with torch.no_grad():
                output = model.generate(**inputs, max_length=1000, do_sample=True, temperature=0.7, top_p=0.9)

            # Decode response
            generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

            if generated_code:
                optimized_solutions.append({
                    "question_id": qid,
                    "optimized_code": generated_code.strip()
                })
                print(f"✅ Successfully optimized question_id {qid} ({index}/{len(codes_data)})")
            else:
                print(f"⚠️ Warning: Empty response for question_id {qid}")

        except Exception as e:
            print(f"❌ Error processing question_id {qid}: {e}")
            continue  # Skip this question and proceed

        # Increment API call counter
        api_calls += 1

        # Sleep after every 14 API calls to avoid rate limits
        if api_calls % 14 == 0:
            print("⏳ API limit reached. Sleeping for 60 seconds...")
            time.sleep(60)

    return optimized_solutions

# Run the function
optimized_results = get_optimized_code()

# Save the results
with open("optimized_codes.json", "w") as f:
    json.dump(optimized_results, f, indent=4)

print("🎉 Optimized code has been saved to optimized_codes.json")


ModuleNotFoundError: No module named 'torch'